In [1]:
import tensorflow as tf
import sys
import random
import time

import labelreg.helpers as helper
import labelreg.networks as network
import labelreg.utils as util
import labelreg.losses as loss


# 0 - get configs
config = helper.ConfigParser(sys.argv, 'training')

# 1 - data
reader_moving_image, reader_fixed_image, reader_moving_label, reader_fixed_label = helper.get_data_readers(
    config['Data']['dir_moving_image'],
    config['Data']['dir_fixed_image'],
    config['Data']['dir_moving_label'],
    config['Data']['dir_fixed_label'])


# 2 - graph
ph_moving_image = tf.placeholder(tf.float32, [config['Train']['minibatch_size']]+reader_moving_image.data_shape+[1])
ph_fixed_image = tf.placeholder(tf.float32, [config['Train']['minibatch_size']]+reader_fixed_image.data_shape+[1])
ph_moving_affine = tf.placeholder(tf.float32, [config['Train']['minibatch_size']]+[1, 12])
ph_fixed_affine = tf.placeholder(tf.float32, [config['Train']['minibatch_size']]+[1, 12])
input_moving_image = util.warp_image_affine(ph_moving_image, ph_moving_affine)  # data augmentation
input_fixed_image = util.warp_image_affine(ph_fixed_image, ph_fixed_affine)  # data augmentation

# predicting ddf
reg_net = network.build_network(network_type=config['Network']['network_type'],
                                minibatch_size=config['Train']['minibatch_size'],
                                image_moving=input_moving_image,
                                image_fixed=input_fixed_image)

# loss
ph_moving_label = tf.placeholder(tf.float32, [config['Train']['minibatch_size']]+reader_moving_image.data_shape+[1])
ph_fixed_label = tf.placeholder(tf.float32, [config['Train']['minibatch_size']]+reader_fixed_image.data_shape+[1])
input_moving_label = util.warp_image_affine(ph_moving_label, ph_moving_affine)  # data augmentation
input_fixed_label = util.warp_image_affine(ph_fixed_label, ph_fixed_affine)  # data augmentation

warped_moving_label = reg_net.warp_image(input_moving_label)  # warp the moving label with the predicted ddf

loss_similarity, loss_regulariser = loss.build_loss(similarity_type=config['Loss']['similarity_type'],
                                                    similarity_scales=config['Loss']['similarity_scales'],
                                                    regulariser_type=config['Loss']['regulariser_type'],
                                                    regulariser_weight=config['Loss']['regulariser_weight'],
                                                    label_moving=warped_moving_label,
                                                    label_fixed=input_fixed_label,
                                                    network_type=config['Network']['network_type'],
                                                    ddf=reg_net.ddf)

train_op = tf.train.AdamOptimizer(config['Train']['learning_rate']).minimize(loss_similarity+loss_regulariser)

# utility nodes - for information only
dice = util.compute_binary_dice(warped_moving_label, input_fixed_label)
dist = util.compute_centroid_distance(warped_moving_label, input_fixed_label)

# 3 - training
num_minibatch = int(reader_moving_label.num_data/config['Train']['minibatch_size'])
train_indices = [i for i in range(reader_moving_label.num_data)]
test_labels = [1,12,18,19,23]
saver = tf.train.Saver(max_to_keep=1)
sess = tf.Session()
sess.run(tf.global_variables_initializer())
for step in range(config['Train']['total_iterations']):

    if step in range(0, config['Train']['total_iterations'], num_minibatch):
        random.shuffle(train_indices)

    minibatch_idx = step % num_minibatch
    case_indices = train_indices[
                    minibatch_idx*config['Train']['minibatch_size']:(minibatch_idx+1)*config['Train']['minibatch_size']]
    #print(case_indices)
    label_indices = [test_labels[random.randint(0,4)] for i in case_indices]
    #print(label_indices)

    trainFeed = {ph_moving_image: reader_moving_image.get_data(case_indices),
                 ph_fixed_image: reader_fixed_image.get_data(case_indices),
                 ph_moving_label: reader_moving_label.get_data(case_indices, label_indices),
                 ph_fixed_label: reader_fixed_label.get_data(case_indices, label_indices),
                 ph_moving_affine: helper.random_transform_generator(config['Train']['minibatch_size']),
                 ph_fixed_affine: helper.random_transform_generator(config['Train']['minibatch_size'])}

    sess.run(train_op, feed_dict=trainFeed)

    if step in range(0, config['Train']['total_iterations'], config['Train']['freq_info_print']):
        current_time = time.asctime(time.gmtime())
        loss_similarity_train, loss_regulariser_train, dice_train, dist_train = sess.run(
            [loss_similarity,
             loss_regulariser,
             dice,
             dist],
            feed_dict=trainFeed)

        # print('----- Training -----')
        print('Step %d [%s]: Loss=%f (similarity=%f, regulariser=%f)' %
              (step,
               current_time,
               loss_similarity_train+loss_regulariser_train,
               1-loss_similarity_train,
               loss_regulariser_train))
        print('  Dice: %s' % dice_train)
        print('  Distance: %s' % dist_train)
        print('  Image-label indices: %s - %s' % (case_indices, label_indices))

    if step in range(0, config['Train']['total_iterations'], config['Train']['freq_model_save']):
        save_path = saver.save(sess, config['Train']['file_model_save'])
        print("Model saved in: %s" % save_path)


Reading default config file in: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\config_demo.ini.

[Data]: dir_moving_image: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\train\moving_images
[Data]: dir_fixed_image: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\train\fixed_images
[Data]: dir_moving_label: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\train\moving_labels
[Data]: dir_fixed_label: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\train\fixed_labels
[Network]: network_type: local
[Loss]: similarity_type: dice
[Loss]: similarity_scales: [0, 1, 2, 4, 8, 16]
[Loss]: regulariser_type: bending
[Loss]: regulariser_weight: 0.5
[Train]: total_iterations: 400
[Train]: learning_rate: 0.0001
[Train]: minibatch_siz

Step 125 [Thu Mar 28 00:08:35 2019]: Loss=0.761377 (similarity=0.239537, regulariser=0.000914)
  Dice: [0.38780805 0.6134176  0.6835443  0.5212903 ]
  Distance: [2.980486  1.4025403 1.3490825 1.63245  ]
  Image-label indices: [101, 146, 74, 27] - [23, 18, 12, 18]
Step 130 [Thu Mar 28 00:11:27 2019]: Loss=0.743143 (similarity=0.258089, regulariser=0.001232)
  Dice: [0.5740822  0.37145558 0.5551269  0.42948514]
  Distance: [2.1151805 3.3044703 2.5953028 2.6476984]
  Image-label indices: [142, 128, 105, 133] - [19, 23, 18, 18]
Step 135 [Thu Mar 28 00:14:20 2019]: Loss=0.757740 (similarity=0.243444, regulariser=0.001184)
  Dice: [0.37642425 0.6401989  0.49072164 0.58048606]
  Distance: [3.529704   1.0320661  2.4359224  0.88168055]
  Image-label indices: [65, 121, 17, 70] - [19, 1, 12, 18]
Step 140 [Thu Mar 28 00:17:16 2019]: Loss=0.704791 (similarity=0.296975, regulariser=0.001766)
  Dice: [0.50535166 0.61221665 0.6723296  0.6484637 ]
  Distance: [1.9471397  1.2765173  0.24792539 1.0125304

Step 275 [Thu Mar 28 01:34:54 2019]: Loss=0.743810 (similarity=0.257254, regulariser=0.001063)
  Dice: [0.72031665 0.7952974  0.51202065 0.49947533]
  Distance: [0.854569   0.48001394 1.6978751  2.8001328 ]
  Image-label indices: [145, 12, 47, 61] - [12, 23, 1, 19]
Step 280 [Thu Mar 28 01:37:45 2019]: Loss=0.753969 (similarity=0.248503, regulariser=0.002473)
  Dice: [0.64378107 0.5240793  0.48884758 0.53822714]
  Distance: [1.7459393 3.1929688 1.3239603 2.6760516]
  Image-label indices: [106, 135, 29, 147] - [23, 12, 19, 18]
Step 285 [Thu Mar 28 01:40:35 2019]: Loss=0.761240 (similarity=0.240174, regulariser=0.001414)
  Dice: [0.6680355  0.61171365 0.48215827 0.3819242 ]
  Distance: [1.3601592 2.4825613 3.0678282 1.8688221]
  Image-label indices: [34, 48, 30, 10] - [19, 12, 1, 12]
Step 290 [Thu Mar 28 01:43:26 2019]: Loss=0.777419 (similarity=0.225179, regulariser=0.002598)
  Dice: [0.4984326  0.6337158  0.67312664 0.3022942 ]
  Distance: [1.5829016 0.7644102 0.5244959 3.5421565]
  Ima

In [1]:
import tensorflow as tf
import sys

import labelreg.helpers as helper
import labelreg.networks as network
import labelreg.apps as app


# 0 - get configs
config = helper.ConfigParser(sys.argv, 'inference')

# 1 - images to register
reader_moving_image, reader_fixed_image, _, _ = helper.get_data_readers(config['Inference']['dir_moving_image'],
                                                                        config['Inference']['dir_fixed_image'])

# 2 - graph
# network for predicting ddf only
ph_moving_image = tf.placeholder(tf.float32, [reader_moving_image.num_data]+reader_moving_image.data_shape+[1])
ph_fixed_image = tf.placeholder(tf.float32, [reader_fixed_image.num_data]+reader_fixed_image.data_shape+[1])

reg_net = network.build_network(network_type=config['Network']['network_type'],
                                minibatch_size=reader_moving_image.num_data,
                                image_moving=ph_moving_image,
                                image_fixed=ph_fixed_image)

# restore the trained weights
saver = tf.train.Saver()
sess = tf.Session()
saver.restore(sess, config['Inference']['file_model_saved'])


# 3 - compute ddf
testFeed = {ph_moving_image: reader_moving_image.get_data(),
            ph_fixed_image: reader_fixed_image.get_data()}
ddf = sess.run(reg_net.ddf, feed_dict=testFeed)
helper.write_images(ddf, config['Inference']['dir_save'], 'ddf')

# warp the test images
warped_images = app.warp_volumes_by_ddf(reader_moving_image.get_data(), ddf)
helper.write_images(warped_images, config['Inference']['dir_save'], 'warped_image')

# warp test labels of gland segmentation, i.e. label_indices=0
if config['Inference']['dir_moving_label']:
    data_moving_label = helper.DataReader(config['Inference']['dir_moving_label'],labelr=True).get_data(label_indices=[1])
    warped_labels = app.warp_volumes_by_ddf(data_moving_label, ddf)
    helper.write_images(warped_labels, config['Inference']['dir_save'], 'warped_label')


Reading default config file in: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\config_demo.ini.

[Network]: network_type: local
[Inference]: file_model_saved: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\saved_model4\model.ckpt
[Inference]: dir_moving_image: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\test\moving_images
[Inference]: dir_fixed_image: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\test\fixed_images
[Inference]: dir_save: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\inference_results\Original4_results
[Inference]: dir_moving_label: C:\Users\rahul\OneDrive\Desktop\Courses\ECE209as\Final Project\label-reg-master\labelreg\ibsr v2\data_final\test\moving_labels
[Inference]: dir_fixed_label: C:\User